In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.manifold import TSNE
import statsmodels.api as sm
from sklearn.decomposition import KernelPCA

In [68]:
expression = pd.read_csv('../P4/Gene_expression.csv')

In [69]:
y = pd.read_csv('labels.csv')
y = y.drop('Unnamed: 0',axis =1 )
y

,patient_id,Label
0,TCGA-05-4244,0
1,TCGA-05-4249,1
2,TCGA-05-4382,0
3,TCGA-05-4384,0
4,TCGA-05-4389,1
...,...,...
402,TCGA-NJ-A55O,0
403,TCGA-NJ-A55R,0
404,TCGA-NJ-A7XG,0
405,TCGA-O1-A52J,1


In [70]:
meta_file = y.merge(expression)
meta_file = meta_file.drop(['patient_id','sample_type'],axis = 1)

In [71]:
meta_file

,Label,?|100133144,?|100134869,?|10357,?|10431,?|155060,?|26823,?|340602,?|388795,?|390284,...,ZWILCH|55055,ZWINT|11130,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009
0,0,0.115255,0.097686,-1.940808,-0.033268,1.057022,1.176377,-0.577514,1.530721,0.136966,...,-0.745395,0.023953,-0.205987,-0.438921,-0.233740,-1.752495,0.980841,-0.505237,-0.279620,-0.447237
1,1,-0.233267,0.219007,-0.289184,0.202607,-0.069052,1.877450,-0.577514,0.029955,0.027188,...,-0.977605,-0.764733,0.780866,0.824841,0.422735,-0.120232,-0.112752,-0.890109,0.314648,0.096801
2,0,0.248854,-0.384813,-0.533564,-0.173685,0.583903,0.236868,1.884460,0.123771,-0.210492,...,0.688824,0.527161,-0.170358,-0.028243,-0.275621,0.608923,0.369427,1.273709,0.374660,-0.019820
3,0,-1.015130,-1.252003,-0.898805,-1.181021,0.757503,0.484314,-0.577514,-0.013923,0.260217,...,-0.410419,-1.600654,1.344431,1.144063,0.782634,-0.680088,-0.150073,-0.171071,1.255729,0.236221
4,1,-0.816478,0.346711,-1.143054,1.242887,-1.570895,-0.056828,-0.577514,-0.166251,-0.193914,...,0.604849,0.750372,1.330608,1.397679,-0.065455,0.673791,-1.420624,-0.711757,-0.833433,0.356558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,0,0.593944,0.996343,-0.009038,0.047639,0.668383,1.340455,-0.577514,-1.208640,-0.104800,...,-0.832927,-0.431204,-0.148779,-0.154760,0.222568,-0.210608,-0.386059,0.289729,0.667480,-0.396243
402,0,-0.069018,-0.011506,-0.817153,-0.316691,1.148248,0.675043,0.645656,2.908490,-1.628148,...,-1.495877,-0.170338,0.329302,0.156824,1.156270,0.476298,-0.242561,-0.339250,1.557140,-0.263652
403,0,1.749735,2.648401,-0.450322,-0.660086,1.016852,-0.892399,-0.577514,0.941106,-1.506687,...,-2.016211,-1.461710,-2.061653,-1.762218,0.543399,-0.541973,-1.441193,-1.226014,0.813458,-1.338024
404,1,0.486836,1.062102,-1.368185,0.780825,0.141787,3.015645,1.099217,0.123525,-0.071799,...,-1.826732,-0.432321,2.537119,1.141717,0.724190,-1.841894,-0.135198,0.430325,0.959205,-1.000029


In [ ]:
meta_file.columns = meta_file.columns.str.replace(r'\|.*', '')

## Find the DE genes 

In [93]:
y = meta_file.iloc[:, 0]  # Target (the first column)
ratios = []
p_values = []
# Iterate over the remaining columns for testing
for column in meta_file.columns[1:]:
    X = meta_file[column]  # Select one feature at a time

    # Add intercept column to the feature
    X = sm.add_constant(X)

    # Perform logistic regression
    logit_model = sm.Logit(y, X)
    result = logit_model.fit()

    # Obtain the odds ratio and p-value
    odds_ratio = result.params[1]  # Get the coefficient for the feature
    p_value = result.pvalues[1]

    # Record the ratio and p-value in the lists
    ratios.append(odds_ratio)
    p_values.append(p_value)

Optimization terminated successfully.
         Current function value: 0.659688
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.659688
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.659688
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.659688
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.659688
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.659688
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.659688
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.659688
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.659688
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.659688
  

Optimization terminated successfully.
         Current function value: 0.690205
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690161
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.689887
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690533
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.687537
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.689716
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.688955
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.689797
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690758
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.691149
  

Optimization terminated successfully.
         Current function value: 0.687885
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690960
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690682
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690114
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690939
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.689594
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.689863
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690484
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.691291
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.684437
  

Optimization terminated successfully.
         Current function value: 0.690482
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.689528
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.691379
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.690893
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.687696
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.689483
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.691236
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.690048
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690810
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690899
  

Optimization terminated successfully.
         Current function value: 0.691357
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.677536
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690894
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.691356
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.691242
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.689830
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.691288
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.689886
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690570
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.686764
  

...

Optimization terminated successfully.
         Current function value: 0.691127
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.689808
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690004
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.691110
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.691388
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.688496
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683208
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.691053
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.686841
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.688591
  

Optimization terminated successfully.
         Current function value: 0.688908
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.691355
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.690796
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.680540
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.691092
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.690835
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.691398
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.690704
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.691111
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.689346
  

Optimization terminated successfully.
         Current function value: 0.690373
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690917
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.689690
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.691384
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.691354
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.689257
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690920
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.689785
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.688273
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.687318
  

Optimization terminated successfully.
         Current function value: 0.690738
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690687
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.689290
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690508
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.691312
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.691198
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.690731
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690924
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.691368
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.689692
  

Optimization terminated successfully.
         Current function value: 0.684146
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.674186
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.691348
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.690978
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.691287
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.684907
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.687790
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690926
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.690810
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.691300
  

In [127]:
feature_stats = [(column, p_value, odds_ratio) for column, p_value, odds_ratio in zip(meta_file.columns[1:], p_values, ratios)]

# Sort the list based on p-values
sorted_by_pvalue = sorted(feature_stats, key=lambda x: x[1])

# Sort the list based on ratios
sorted_by_ratio = sorted(feature_stats, key=lambda x: abs(x[2]), reverse=True)

# # Print the sorted lists
# print("Features sorted by p-values:")
# for feature in sorted_by_pvalue:
#     print(f"Feature: {feature[0]}, P-value: {feature[1]}, Ratio: {feature[2]}")

# print("\nFeatures sorted by ratios:")
# for feature in sorted_by_ratio:
#     print(f"Feature: {feature[0]}, P-value: {feature[1]}, Ratio: {feature[2]}")

In [111]:
sorted_by_pvalue = sorted(feature_stats, key=lambda x: x[1])

# Retrieve the first 100 features
top_100_features = [feature[0] for feature in sorted_by_pvalue[:100]]

In [113]:
diff_meta_file = meta_file[top_100_features]

In [118]:
X_train, X_test, y_train, y_test = train_test_split(diff_meta_file, meta_file.iloc[:,0], test_size=0.2, random_state=42)

In [119]:
model = LogisticRegression(C = 0.1,penalty='l1',solver='liblinear')

In [120]:
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print('Accurary:')
print(accuracy)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

Accurary:
0.6585365853658537
Confusion Matrix:
[[24 12]
 [16 30]]


## CV

In [121]:
X_train,X_test, y_train, y_test = train_test_split(diff_meta_file, prog_meta_file.iloc[:,0], test_size=0.2, random_state=42)

In [128]:
model = svm.SVC()
param_grid = {'C': [0.1,0.5,1,2,5,10],
              'kernel': ['linear','rbf',]}

In [135]:
model = RandomForestClassifier()

In [136]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10]
}

# Perform grid search cross-validation
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1')
grid_search.fit(X, y)


GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='f1')

In [137]:
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'max_depth': 5, 'n_estimators': 200}
Best Score: 0.5614203378003355


In [138]:
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print('Accurary:')
print(accuracy)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

Accurary:
0.6707317073170732
Confusion Matrix:
[[24 12]
 [15 31]]
